In [2]:
import numpy as np
import pandas as pd
import os
from torchvision import datasets, transforms
import torch
from PIL import Image

In [3]:
base_path = f"{os.path.expanduser('~')}/Downloads/"
if not os.path.exists(base_path):
    os.makedirs(base_path)

is_transform=False

In [4]:
def unpickle(file):
    import pickle

    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


transform = transforms.Compose(
    [transforms.ToTensor(),
     ]) if not is_transform else transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)


# construct test set
test_dict = unpickle(base_path + "cifar-10-batches-py/test_batch")


data = test_dict[b"data"]
label = test_dict[b"labels"]

# from https://stackoverflow.com/questions/55319949/pil-typeerror-cannot-handle-this-data-type
img = np.array(data).astype(np.uint8).reshape(-1, 3, 32, 32).transpose((0, 2, 3, 1))

converted_data = []
for i in img:
    converted_i = Image.fromarray(i)
    converted_i = transform(converted_i).reshape(3072)
    converted_data.append(converted_i)

data = np.stack(converted_data)
label = np.array(label).reshape((len(label), 1))

test_data = np.concatenate((data, label), axis=1)
print(f"Test shape: {test_data.shape}")
print(f"Test: {test_data[0]}")


test_df = pd.DataFrame(test_data)
# test_df = test_df.sample(frac=1)
# test_df.to_csv(base_path + "cifar10_test.csv", index=False, header=True)

# construct training set
batch_list = []
for i in range(1, 6):
    test_dict = unpickle(f"{base_path}cifar-10-batches-py/data_batch_{i}")

    data = test_dict[b"data"]
    label = test_dict[b"labels"]

    # from https://stackoverflow.com/questions/55319949/pil-typeerror-cannot-handle-this-data-type
    img = np.array(data).astype(np.uint8).reshape(-1, 3, 32, 32).transpose((0, 2, 3, 1))

    converted_batch = []
    for i in img:
        converted_i = Image.fromarray(i)
        converted_i = transform(converted_i).reshape(3072)
        converted_batch.append(converted_i)

    data = np.stack(converted_batch)
    data = np.array(data)
    label = np.array(label).reshape((len(label), 1))

    merge_data = np.concatenate((data, label), axis=1)

    batch_list.append(merge_data)

train_data = np.concatenate(batch_list)
print(f"Train shape: {train_data.shape}")
print(f"Train: {train_data[0]}")

train_df = pd.DataFrame(train_data)
# train_df = train_df.sample(frac=1)
# train_df.to_csv(base_path + "cifar10_train.csv", index=False, header=True)


Test shape: (10000, 3073)
Test: [0.61960787 0.62352943 0.64705884 ... 0.50588238 0.43137255 3.        ]
Train shape: (50000, 3073)
Train: [0.23137255 0.16862746 0.19607843 ... 0.32941177 0.28235295 6.        ]


In [5]:
import numpy as np
import pandas as pd
import os
from torchvision import datasets, transforms
import torch
from PIL import Image

In [6]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      ])

batch_size = 32

path = f'{os.path.expanduser("~")}/Downloads'
trainset = datasets.CIFAR10(root=path, train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=False)

testset = datasets.CIFAR10(root=path, train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [43]:
for i, (img, label) in enumerate(testloader):
    assert torch.equal(label, torch.tensor(test_data[i*batch_size:(i+1)*batch_size, -1]))
    # print(img.reshape(-1, 32*32*3).to(torch.float64))
    # print(torch.tensor(test_data[i*batch_size:(i+1)*batch_size, :-1]))
    # print(torch.allclose(img.reshape(-1, 32*32*3).to(torch.float64), torch.tensor(test_data[i*batch_size:(i+1)*batch_size, :-1])))
    assert torch.allclose(img.reshape(-1, 32*32*3).to(torch.float64), torch.tensor(test_data[i*batch_size:(i+1)*batch_size, :-1]))
    # assert label == torch.tensor(test_data[i*batch_size:(i+1)*batch_size, -1])

for i, (img, label) in enumerate(trainloader):
    assert torch.equal(label, torch.tensor(train_data[i*batch_size:(i+1)*batch_size, -1]))
    assert torch.allclose(img.reshape(-1, 32*32*3).to(torch.float64), torch.tensor(train_data[i*batch_size:(i+1)*batch_size, :-1]))

    # assert label == torch.tensor(test_data[i*batch_size:(i+1)*batch_size, -1])

## Validate outputs during training

In [3]:
import torch
plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-log.pt")
print(plaintext_outputs[0].shape)

iter = 10000
batch_size = 32

secure_outputs = []
# load secure training
with open(f"{os.path.expanduser('~')}/DNN/output/AlexNet preloaded init weights train_SPDZ_GPU_Full_CIFAR10_32_4_CE_debug_{iter}.txt", 'r') as secF:
    for line in secF:
        row_data = []
        for t in line.strip().split(" "):
            row_data.append(float(t))
        secure_outputs.append(torch.tensor(row_data).reshape(batch_size, 10))

print(secure_outputs[0].shape)

print(len(plaintext_outputs))
print(len(secure_outputs))

length = min(len(plaintext_outputs), len(secure_outputs))
length = 0

for i in range(length):
    diff = plaintext_outputs[i].clone().detach() - secure_outputs[i]
    print(f'Iteration: {i}, max absoulte error: {torch.abs(diff).max()}')
    print(f'Iteration: {i}, plain: {plaintext_outputs[i].clone().detach()[0]}, secure: {secure_outputs[i][0]}')
    where_diff = torch.abs(diff) > 1e-3
    # where_diff = torch.isclose(plaintext_outputs[i].clone().detach(), secure_outputs[i], 1e-3)
    # print(torch.masked_select(diff, where_diff))
    print(torch.allclose(plaintext_outputs[i].clone().detach(), secure_outputs[i], atol=1e-2))
    # print(plaintext_outputs[i].clone().detach())
    # print(secure_outputs[i])

# print("Plaintext: ", plaintext_outputs[-1].clone().detach().numpy())

for i, o in enumerate(secure_outputs):
    print(f'Secure outputs-{i}: {o[0]}')
# print((plaintext_outputs[0].clone().detach()[0][0].numpy(), secure_outputs[0][0][0].numpy()))
# print(torch.allclose(plaintext_outputs[0].clone().detach()[0][0], secure_outputs[0][0][0], atol=1e-3))


torch.Size([32, 10])
torch.Size([32, 10])
1500
16000
Secure outputs-0: tensor([-0.0289,  0.0126, -0.0206,  0.0436,  0.0224, -0.0137,  0.0404, -0.0213,
        -0.0337,  0.0606])
Secure outputs-1: tensor([-0.0312,  0.0093, -0.0181,  0.0458,  0.0248, -0.0198,  0.0453, -0.0212,
        -0.0391,  0.0670])
Secure outputs-2: tensor([-0.0348,  0.0167, -0.0041,  0.0499,  0.0206, -0.0200,  0.0355, -0.0224,
        -0.0448,  0.0637])
Secure outputs-3: tensor([-0.0383,  0.0219, -0.0103,  0.0517,  0.0234, -0.0199,  0.0318, -0.0167,
        -0.0476,  0.0652])
Secure outputs-4: tensor([-0.0411,  0.0323, -0.0081,  0.0454,  0.0172, -0.0263,  0.0366, -0.0198,
        -0.0479,  0.0719])
Secure outputs-5: tensor([-0.0462,  0.0350, -0.0113,  0.0467,  0.0219, -0.0239,  0.0383, -0.0206,
        -0.0474,  0.0685])
Secure outputs-6: tensor([-0.0437,  0.0367, -0.0172,  0.0432,  0.0216, -0.0186,  0.0352, -0.0149,
        -0.0474,  0.0677])
Secure outputs-7: tensor([-0.0415,  0.0354, -0.0171,  0.0423,  0.0122, -